## Yfinance data fetcher Async
    Matheus Laidler

A biblioteca yfinance é muito útil, mas para quem visa buscar dados de inúmeros ativos simultaneamente, pode ser meio lento. O que ocorre é que a biblioteca yfinance é sincrono, o que faz com que cada request dependa da resposta chegar ao nosso computador antes de ser inicializada. Aqui desenvolvo uma versão assíncrona, para que possamos buscar dados de forma paralela, o que acelera muito a velocidade do nosso código.

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
import multiprocessing as mp
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import poisson
from scipy.optimize import curve_fit

In [18]:
import asyncio
import aiohttp
from io import StringIO
import datetime as dt
from dateutil.relativedelta import relativedelta
from pytz import utc

class yfinance_async:
    """
    Asynchronous class for fetching historical financial data from Yahoo Finance.

    This class enables asynchronous fetching of financial data for a specified ticker symbol.

    Args:
        ticker (str): The ticker symbol for the asset of interest. The ticker is different then the one the show on the website,
        its necessary to get the url of the download button the verify it.

    Attributes:
        ticker (str): The ticker symbol associated with the asset.
        base_url (str): The base URL used to construct the API endpoint for data retrieval.
        header (dict): HTTP headers to be included in the API request, preventing eventual bot interruption.
        params (dict): Parameters to customize the data retrieval (e.g., time interval).

    Methods:
        async def fetch(self):
            Asynchronously fetches raw data from the Yahoo Finance API.

        async def get_data(self):
            Asynchronously retrieves and parses data into a pandas DataFrame.

    Usage:
        ticker = 'AAPL'
        yf = yfinance_async(ticker)
        df = await yf.get_data()
    """

    def __init__(self):
        self.base_url = f"https://query1.finance.yahoo.com/v7/finance/download/"
        self.header = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.57"
        }
        self.params = {
            "period1": int(
                (dt.datetime.now(tz=utc) - relativedelta(years=40)).timestamp()
            ),  # Default: fetch real time -> 1day data
            "period2": int(dt.datetime.now(tz=utc).timestamp()),
            "interval": "1d",
            "events": "history",
        }

    async def fetch(self, ticker: str):
        url = f"{self.base_url}{ticker}?"
        async with aiohttp.ClientSession() as session:
            async with session.get(
                url, params=self.params, headers=self.header
            ) as response:
                return await response.text()

    async def get_data(self, ticker: str):
        response_text = await self.fetch(ticker=ticker)
        df = pd.read_csv(StringIO(response_text))
        return df

In [22]:
yahoo_data = yfinance_async()

In [25]:
df = await yahoo_data.get_data(ticker = 'BTC')